In [25]:
'''import all necessary libraries'''

import cx_Oracle #import the oracle database libraries
import pandas as pd #connect to the pandas libraries
import numpy as np #import numpy
import getpass #import the password libraries in order to keep database password concealed

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import sys
print (sys.version)
pd.options.display.float_format = '{:0.6f}'.format

3.6.5 |Anaconda custom (64-bit)| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]


In [2]:
'''connect to the oracle server used to pull down the 
currency and misuse of authorization information'''

username=input("please enter your username")
password=getpass.getpass("Enter your password: ")

con=cx_Oracle.connect(username,password,'ODSPEU1.ams') #enter database details
print (con.version) # prnt the conversion details

please enter your username djohnson
Enter your password:  ··········


12.1.0.2.0


In [3]:
'''run query from IEP database'''

df1=pd.read_sql(
''' 
select BC.DATA_MONTH,BC.GROUP_NUMBER,BC.STATEMENT_NARRATIVE,BC.ISSUER_REGION_DESCRIPTION,BC.CREDIT___DEBIT,BC.CARD_SCHEME_DESCRIPTION
,SUM(BC.NUMBER_OF_SALES) AS VOLUME,SUM(BC.FLOW_EUR_TYPE_SPECIFIC) AS FLOW_EUR,SUM(BC.SCHEME_FEE_EUR) AS SF_EUR
from FDWO.ACQUIRER_BARCLAYS_COST_2017 bc
where BC.COST_TYPE='SCHEME FEE'
and BC.TRANSACTION_TYPE='TRANSACTION'
and BC.VALUE_OF_SALES>0
group by BC.DATA_MONTH,BC.GROUP_NUMBER,BC.STATEMENT_NARRATIVE,BC.ISSUER_REGION_DESCRIPTION,BC.CREDIT___DEBIT,BC.CARD_SCHEME_DESCRIPTION
''', con)

In [14]:
df2=pd.read_csv('G:\\Pricing\\Pricing Intelligence\\Market Changes\\Pricing Updates & Bulletins\\Scheme Fees\\Partners\\Barclays\\2018\\barclays_2018_fee_schedule_for_python.csv')

In [15]:
df1_working=df1.copy()
df2_working=df2[['GROUP_NUMBER','CREDIT___DEBIT','ISSUER_REGION_DESCRIPTION','CARD_SCHEME_DESCRIPTION','PERC_RATE','FIXED_RATE']].copy()

In [16]:
df1_working.STATEMENT_NARRATIVE.unique()

array(['Scheme Fee Intra Maestro', 'Scheme Fee Intra VISA Credit',
       'Scheme Fee Inter MC Debit', 'Scheme Fee Intra MC Credit',
       'Scheme Fee Intra VISA Debit', 'Scheme Fee Intra VISA Vpay',
       'Scheme Fee Domestic VISA Credit', 'Scheme Fee Inter VISA Debit',
       'Scheme Fee Intra MC Debit', 'Scheme Fee Domestic MC Credit',
       'Scheme Fee Inter MC Credit', 'Scheme Fee Inter VISA Credit',
       'Scheme Fee Domestic Maestro', 'Scheme Fee Domestic VISA Debit',
       'Scheme Fee Domestic VISA Vpay', 'Scheme Fee Domestic MC Debit',
       'Scheme Fee Inter Maestro'], dtype=object)

In [17]:
df1_working['GROUP_NUMBER']=df1_working['GROUP_NUMBER'].astype(float)
df2_working['GROUP_NUMBER']=df2_working['GROUP_NUMBER'].astype(float)

In [18]:
for x in range(0,len(df1_working)):
    if 'VISA Credit' in df1_working.loc[x,'STATEMENT_NARRATIVE']:
        df1_working.loc[x,'CREDIT___DEBIT']='CR'
    if 'MC Credit' in df1.loc[x,'STATEMENT_NARRATIVE']:
        df1_working.loc[x,'CREDIT___DEBIT']='CR'

In [19]:
df1_working.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 975 entries, 0 to 974
Data columns (total 9 columns):
DATA_MONTH                   975 non-null datetime64[ns]
GROUP_NUMBER                 975 non-null float64
STATEMENT_NARRATIVE          975 non-null object
ISSUER_REGION_DESCRIPTION    975 non-null object
CREDIT___DEBIT               975 non-null object
CARD_SCHEME_DESCRIPTION      975 non-null object
VOLUME                       975 non-null int64
FLOW_EUR                     975 non-null float64
SF_EUR                       975 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 68.6+ KB


In [20]:
df2_working.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 6 columns):
GROUP_NUMBER                 152 non-null float64
CREDIT___DEBIT               152 non-null object
ISSUER_REGION_DESCRIPTION    152 non-null object
CARD_SCHEME_DESCRIPTION      152 non-null object
PERC_RATE                    112 non-null float64
FIXED_RATE                   112 non-null float64
dtypes: float64(3), object(3)
memory usage: 7.2+ KB


In [21]:
master_data_set=pd.merge(df1_working,df2_working,how='left',on=['GROUP_NUMBER','CREDIT___DEBIT','ISSUER_REGION_DESCRIPTION','CARD_SCHEME_DESCRIPTION'])

In [26]:
master_data_set.head()

,DATA_MONTH,GROUP_NUMBER,STATEMENT_NARRATIVE,ISSUER_REGION_DESCRIPTION,CREDIT___DEBIT,CARD_SCHEME_DESCRIPTION,VOLUME,FLOW_EUR,SF_EUR,PERC_RATE,FIXED_RATE,THEORETICAL_SF
0,2017-10-01,8765.000000,Scheme Fee Intra Maestro,INTRA,DR,MasterCard,87,609.499600,4.900000,0.000470,0.002080,0.467425
1,2017-10-01,8765.000000,Scheme Fee Intra VISA Credit,INTRA,CR,Visa,43796,703577.992300,337.710000,0.000240,0.016790,904.193558
2,2017-10-01,8765.000000,Scheme Fee Inter MC Debit,INTRA,DR,MasterCard,39,758.928800,8.690000,0.000470,0.002080,0.437817
3,2017-10-01,8765.000000,Scheme Fee Intra MC Credit,INTRA,CR,MasterCard,495172,1772035.739500,59115.700000,0.000470,0.002080,1862.814558
4,2017-10-01,8765.000000,Scheme Fee Intra VISA Debit,INTRA-E,DR,Visa,4612,47287.144600,27.640000,0.000200,0.016790,86.892909


In [23]:
master_data_set['THEORETICAL_SF']=(master_data_set['FLOW_EUR']*master_data_set['PERC_RATE'])+(master_data_set['VOLUME']*master_data_set['FIXED_RATE'])

In [29]:
master_data_set.head()

,DATA_MONTH,GROUP_NUMBER,STATEMENT_NARRATIVE,ISSUER_REGION_DESCRIPTION,CREDIT___DEBIT,CARD_SCHEME_DESCRIPTION,VOLUME,FLOW_EUR,SF_EUR,PERC_RATE,FIXED_RATE,THEORETICAL_SF
0,2017-10-01,8765.000000,Scheme Fee Intra Maestro,INTRA,DR,MasterCard,87,609.499600,4.900000,0.000470,0.002080,0.467425
1,2017-10-01,8765.000000,Scheme Fee Intra VISA Credit,INTRA,CR,Visa,43796,703577.992300,337.710000,0.000240,0.016790,904.193558
2,2017-10-01,8765.000000,Scheme Fee Inter MC Debit,INTRA,DR,MasterCard,39,758.928800,8.690000,0.000470,0.002080,0.437817
3,2017-10-01,8765.000000,Scheme Fee Intra MC Credit,INTRA,CR,MasterCard,495172,1772035.739500,59115.700000,0.000470,0.002080,1862.814558
4,2017-10-01,8765.000000,Scheme Fee Intra VISA Debit,INTRA-E,DR,Visa,4612,47287.144600,27.640000,0.000200,0.016790,86.892909


In [31]:
master_data_set.to_csv('d:\\djohnson\\Desktop\\barclays_sf_analysis\\barclays_sf_analysis.csv')